In [1]:
import boto3
import os

In [2]:
dynamo_client = boto3.client('dynamodb')

In [3]:
dynamo_client.list_tables()

{'TableNames': ['jobs', 'jobs_run_details'],
 'ResponseMetadata': {'RequestId': '9IOO5Q3JFKHGFODKM6G6VSUPJ7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 27 Aug 2023 14:12:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '42',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9IOO5Q3JFKHGFODKM6G6VSUPJ7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3921917396'},
  'RetryAttempts': 0}}

In [4]:
tables = dynamo_client.list_tables()['TableNames']


In [5]:
'jobs' in tables

True

In [6]:
if 'jobs' in tables:
    response = dynamo_client.create_table(
        AttributeDefinitions =[
            {
                'AttributeName':'job_id',
                'AttributeType':'S'
            },
            {
                'AttributeName':'job_run_time',
                'AttributeType':'N'
            }
        ],
        TableName='jobs_run_details',
        KeySchema=[
            {
            'AttributeName':'job_id',
            'KeyType': 'HASH'
            },
            {
            'AttributeName':'job_run_time',
            'KeyType': 'RANGE'
            },
            
        ],
        BillingMode='PAY_PER_REQUEST'
        
    )
    print(response)
    


ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: jobs_run_details

In [7]:
tables = dynamo_client.list_tables()['TableNames']
tables

['jobs', 'jobs_run_details']

In [8]:
dynamo_client.describe_table(TableName='jobs_run_details')


{'Table': {'AttributeDefinitions': [{'AttributeName': 'job_id',
    'AttributeType': 'S'},
   {'AttributeName': 'job_run_time', 'AttributeType': 'N'}],
  'TableName': 'jobs_run_details',
  'KeySchema': [{'AttributeName': 'job_id', 'KeyType': 'HASH'},
   {'AttributeName': 'job_run_time', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 8, 27, 19, 27, 26, 18000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:130663382464:table/jobs_run_details',
  'TableId': '7190e4fc-ddae-46cb-8a36-26e007c23fe9',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2023, 8, 27, 19, 27, 26, 18000, tzinfo=tzlocal())}},
 'ResponseMetadata': {'RequestId': 'EE6KJRUP2PQEOKC6MS4AR3UCC7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,

In [9]:
dynamodb_resource = boto3.resource('dynamodb')

job_details_table = dynamodb_resource.Table('jobs_run_details')


In [10]:
type(job_details_table)

boto3.resources.factory.dynamodb.Table

In [11]:
import time
import datetime


In [12]:
int(time.mktime(datetime.datetime.now().timetuple()))

1693145599

In [13]:
job_details = {
    'job_id': 'ghactivity_ingest',
    'job_run_time': int(time.mktime(datetime.datetime.now().timetuple())),
    'job_run_bookmark_details':{
        'processed_file_name':'2022-06-03-0.json.gz'
    }
}

In [14]:
# insert the record into tables
job_details_table.put_item(
    Item = job_details
)

{'ResponseMetadata': {'RequestId': '3JLTK0SELOTH3MSGI617NJQGLNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 27 Aug 2023 14:13:24 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3JLTK0SELOTH3MSGI617NJQGLNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [33]:
job_details_table.scan() # returns all the rows in table

{'Items': [{'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'processed_file_name': '2022-06-03-0.json.gz'},
   'job_run_time': Decimal('1691673936')}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'JUO6FPPJGAF79U92VLDHPH65QBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 10 Aug 2023 13:25:43 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '195',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'JUO6FPPJGAF79U92VLDHPH65QBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3728954201'},
  'RetryAttempts': 0}}

In [15]:
# get item return only one row based on the key passed
job_details_table.get_item(Key={'job_id':'ghactivity_ingest'})


ClientError: An error occurred (ValidationException) when calling the GetItem operation: The provided key element does not match the schema

In [13]:
# Delete the table from dynamoDB

dynamo_client.delete_table(TableName='jobss')
waiter = dynamo_client.get_waiter('table_not_exists')
waiter.wait(TableName='jobss')
print ("table deleted")

table deleted
